# Simple Reinforcement Learning with Tensorflow Part 4: Deep Q-Networks and Beyond

In this iPython notebook I implement a Deep Q-Network using both Double DQN and Dueling DQN. The agent learn to solve a navigation task in a basic grid world. To learn more, read here: https://medium.com/p/8438a3e2b8df

For more reinforcment learning tutorials, see:
https://github.com/awjuliani/DeepRL-Agents

In [2]:
from __future__ import division
from lib.SQL import SQLCalls
from PIL import ImageGrab
import gym
import numpy as np
import random
import tensorflow as tf
import tensorflow.contrib.slim as slim
import matplotlib.pyplot as plt
import scipy.misc
import cv2
import os
%matplotlib inline
SQL=SQLCalls()
img=ImageGrab.grab(bbox=(0,60,580,530))
Genomes=SQL.GatherGenomes()
POPULATION=len(Genomes)
UsedGenomes=np.ones(Genomes.shape[0])

Using TensorFlow backend.


### Load the game environment

Feel free to adjust the size of the gridworld. Making it smaller provides an easier task for our DQN agent, while making the world larger increases the challenge.

Above is an example of a starting environment in our simple game. The agent controls the blue square, and can move up, down, left, or right. The goal is to move to the green square (for +1 reward) and avoid the red square (for -1 reward). The position of the three blocks is randomized every episode.

### Implementing the network itself

### Experience Replay

This class allows us to store experies and sample then randomly to train the network.

In [3]:
class experience_buffer():
    def __init__(self, buffer_size = 50000):
        self.buffer = []
        self.buffer_size = buffer_size
    
    def add(self,experience):
        if len(self.buffer) + len(experience) >= self.buffer_size:
            self.buffer[0:(len(experience)+len(self.buffer))-self.buffer_size] = []
        self.buffer.extend(experience)
            
    def sample(self,size):
        return np.reshape(np.array(random.sample(self.buffer,size)),[size,5])

This is a simple function to resize our game frames.

In [5]:
def processState(states):
    return np.reshape(states,[21168])

These functions allow us to update the parameters of our target network with those of the primary network.

In [6]:
def updateTargetGraph(tfVars,tau):
    total_vars = len(tfVars)
    op_holder = []
    for idx,var in enumerate(tfVars[0:total_vars//2]):
        op_holder.append(tfVars[idx+total_vars//2].assign((var.value()*tau) + ((1-tau)*tfVars[idx+total_vars//2].value())))
    return op_holder

def updateTarget(op_holder,sess):
    for op in op_holder:
        sess.run(op)
        
#https://gist.github.com/danijar/3f3b547ff68effb03e20c470af22c696
#https://danijar.com/variable-sequence-lengths-in-tensorflow/
def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length
#https://gist.github.com/danijar/3f3b547ff68effb03e20c470af22c696
#https://danijar.com/variable-sequence-lengths-in-tensorflow/
def last_relevant(output, length):
    batch_size = tf.shape(output)[0]
    max_length = int(output.get_shape()[1])
    output_size = int(output.get_shape()[2])
    index = tf.range(0, batch_size) * max_length + (length - 1)
    flat = tf.reshape(output, [-1, output_size])
    relevant = tf.gather(flat, index)
    return relevant

### Training the network

Setting all the training parameters

In [7]:
batch_size = 16 #How many experiences to use for each training step.
update_freq = 4 #How often to perform a training step.
y = .99 #Discount factor on the target Q-values
startE = 1 #Starting chance of random action
endE = 0.1 #Final chance of random action
anneling_steps = 10000. #How many steps of training to reduce startE to endE.
num_episodes = 10000 #How many episodes of game environment to train network with.
pre_train_steps = 10000 #How many steps of random actions before training begins.
max_epLength = 50 #The max allowed length of our episode.
load_model = False #Whether to load a saved model.
path = "./dqn" #The path to save our model to.
h_size = 1024 #The size of the final convolutional layer before splitting it into Advantage and Value streams.
tau = 0.001 #Rate to update target network toward primary network

In [11]:
def process_img(original_image):
    processed_img= cv2.resize(original_image,(84,84))
    return np.array(np.reshape(processed_img,[84,84,3]))
tf.reset_default_graph()
mainQN = Qnetwork(h_size,84,POPULATION,batch_size,"Main")
targetQN = Qnetwork(h_size,84,POPULATION,batch_size,"Target")

init = tf.global_variables_initializer()

saver = tf.train.Saver()

trainables = tf.trainable_variables()

targetOps = updateTargetGraph(trainables,tau)

myBuffer = experience_buffer()

#Set the rate of random action decrease. 
e = startE
stepDrop = (startE - endE)/anneling_steps

#create lists to contain total rewards and steps per episode
jList = []
rList = []
total_steps = 0

#Make a path for our model to be saved in.
if not os.path.exists(path):
    os.makedirs(path)

with tf.Session() as sess:
    sess.run(init)
    if load_model == True:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(path)
        saver.restore(sess,ckpt.model_checkpoint_path)
    #for i in range(update_freq+1+pre_train_steps):
    for i in range(1):
        episodeBuffer = experience_buffer()
        #Reset environment and get first new observation
        print_screen = np.array(ImageGrab.grab(bbox=(0,60,580,530)))
        new_screen=process_img(print_screen)
        s = processState(new_screen)
        d = False
        rAll = 0
        j = 0
        #The Q-Network
        while j < 1: #If the agent takes longer than 200 moves to reach either of the blocks, end the trial.
            j+=1
            #Choose an action by greedily (with e chance of random action) from the Q-network
#             f_dict={mainQN.used_genomes:UsedGenomes,mainQN.genomes:Genomes,mainQN.scalarInput:[s]}
#             a = sess.run([mainQN.predict],feed_dict=f_dict)
#             print(a)
#             r=random.randint(-1,1)
#             s1 = processState(s1)
#             total_steps += 1
#             episodeBuffer.add(np.reshape(np.array([s,a,r,s1,d]),[1,5])) #Save the experience to our episode buffer.
            
#            if total_steps > pre_train_steps:
#                 if e > endE:
#                     e -= stepDrop
                
#                 if total_steps % (update_freq) == 0:
            trainBatch=SQL.gain_history()
            np.random.shuffle(trainBatch)
            trainBatch=trainBatch[0:batch_size]
            states=np.vstack(trainBatch[:,0])
            states=np.reshape(states,[batch_size,84,84,3])
            states_after=np.vstack(trainBatch[:,3])
            states_after=np.reshape(states,[batch_size,84,84,3])
            UsedGenomes=np.zeros(Genomes.shape[0])
             #Get a random batch of experiences.
                    #Below we perform the Double-DQN update to the target Q-values
            action_list=[]
            answer_list=[]
            for k in range(batch_size):
                m_dict={mainQN.used_genomes:UsedGenomes,mainQN.genomes:Genomes,
                        mainQN.imageIn:[states_after[k]],mainQN.condition:0,
                        mainQN.correct_action:[10],mainQN.correct_mean:[10]}
                t_dict={targetQN.used_genomes:UsedGenomes,targetQN.genomes:Genomes,
                        targetQN.imageIn:[states_after[k]],targetQN.condition:0,
                        targetQN.correct_action:[10],targetQN.correct_mean:[10]}
                Q1 = sess.run(mainQN.predict,feed_dict=m_dict)
                action_list.append(Q1)
                mean,Q2,Value = sess.run([targetQN.Mean,targetQN.Qout,targetQN.Value],feed_dict=t_dict)
                answer_list.append(Q2[0][Q1])
                #mean_list.append(mean[0])
                #print(Value)
            
            #print(answer_list)
            #print(action_list)
            UsedGenomes=np.zeros(batch_size)
            #t_dict={targetQN.used_genomes:UsedGenomes,targetQN.genomes:Genomes,
             #           targetQN.imageIn:states_after,targetQN.condition:2,
             #       targetQN.correct_action:action_list,targetQN.correct_mean:mean_list}
            #Q2A,Q2V,Q2M,Q2=sess.run([targetQN.Advantage,targetQN.Value,targetQN.correct_mean,targetQN.Qout],feed_dict=t_dict)
#             Q2A=np.array(Q2A)
#             Q2V=np.array(Q2V)
#             Q2M=np.array(Q2M)
#             print(Q2V)
#             print(Q2A)
#             print(Q2M)
#             print(Q2A-Q2M+Q2V)
            #print(Q2)
            #print(Q2[range(batch_size),action_list]) 
            #end_multiplier = -(trainBatch[:,4] - 1)
            doubleQ = np.array(answer_list)
            targetQ = trainBatch[:,2] + (y*doubleQ * 1)
            mean_list=[]
            #Update the network with our target values.
            for k in range(batch_size):
                m_dict={mainQN.used_genomes:UsedGenomes,mainQN.genomes:Genomes,
                        mainQN.imageIn:[states[k]],mainQN.condition:0,
                        mainQN.correct_action:[10],mainQN.correct_mean:[10]}
                mean = sess.run([mainQN.Mean],feed_dict=m_dict)
                mean_list.append(mean[0])
            final_dict={mainQN.used_genomes:UsedGenomes,mainQN.genomes:Genomes,
                        mainQN.imageIn:states,mainQN.condition:1,
                        mainQN.correct_action:trainBatch[:,1],mainQN.correct_mean:np.hstack(mean_list),
                       mainQN.targetQ:targetQ}
            _,loss =sess.run([mainQN.updateModel,mainQN.loss], \
                feed_dict=final_dict)
            print(loss)
            #_ = sess.run(mainQN.updateModel, \
             #   feed_dict={mainQN.scalarInput:np.vstack(trainBatch[:,0]),mainQN.targetQ:targetQ, mainQN.actions:trainBatch[:,1]})

            updateTarget(targetOps,sess) #Update the target network toward the primary network.
#             rAll += r
              #s = s1
            
#             if d == True:

#                 break
        
#         myBuffer.add(episodeBuffer.buffer)
#         jList.append(j)
#         rList.append(rAll)
#         #Periodically save the model. 
#         if i % 1000 == 0:
#             saver.save(sess,path+'/model-'+str(i)+'.cptk')
#             print("Saved Model")
#         if len(rList) % 10 == 0:
#             print(total_steps,np.mean(rList[-10:]), e)
#     saver.save(sess,path+'/model-'+str(i)+'.cptk')
# print("Percent of succesful episodes: " + str(sum(rList)/num_episodes) + "%")

Tensor("Placeholder_2:0", shape=(?,), dtype=float32)
Tensor("Placeholder_9:0", shape=(?,), dtype=float32)
31.6679


In [10]:
class Qnetwork():
    def __init__(self,h_size,s_size,POPULATION,BATCH,myScope):
        #The network recieves a frame from the game, flattened into an array.
        #It then resizes it and processes it through four convolutional layers.
        self.genomes= tf.placeholder(shape=[POPULATION,30,3],dtype=tf.int32)   
        self.condition = tf.placeholder(tf.int32, shape=[], name="condition")
        self.correct_action=tf.placeholder(shape=[None], dtype=tf.int32)
        self.correct_mean=tf.placeholder(shape=[None], dtype=tf.float32)
        print(self.correct_mean)
        self.imageIn = tf.placeholder(shape=[None,s_size,s_size,3],dtype=tf.float32)      
        self.used_genomes= tf.placeholder(shape=[None],dtype=tf.int32)
        self.conv1 = slim.conv2d( \
        inputs=self.imageIn,num_outputs=32,kernel_size=[8,8],stride=[4,4],padding='VALID', biases_initializer=None)
        self.conv2 = slim.conv2d( \
            inputs=self.conv1,num_outputs=64,kernel_size=[4,4],stride=[2,2],padding='VALID', biases_initializer=None)
        self.conv3 = slim.conv2d( \
            inputs=self.conv2,num_outputs=64,kernel_size=[3,3],stride=[1,1],padding='VALID', biases_initializer=None)
        self.conv4 = slim.conv2d( \
            inputs=self.conv3,num_outputs=h_size,kernel_size=[7,7],stride=[1,1],padding='VALID', biases_initializer=None)
        #We take the output from the final convolutional layer and split it into separate advantage and value streams.
        self.streamAC,self.streamVC = tf.split(self.conv4,2,3)
        self.streamA = tf.reshape(self.streamAC,[-1,512])
        self.streamV = slim.flatten(self.streamVC)
        xavier_init = tf.contrib.layers.xavier_initializer()
        hidden_conv = slim.fully_connected(self.streamA,h_size//2,biases_initializer=None,activation_fn=tf.nn.relu)
        self.VW=tf.Variable(xavier_init([h_size//2,1]))
        self.Value =tf.cond(self.condition <  1, 
                             lambda: tf.matmul(self.streamV,self.VW), 
                             lambda: tf.reshape(tf.matmul(self.streamV,self.VW),[1,BATCH])[0])
        hidden_conv = tf.cond(self.condition <  1, lambda: tf.tile(hidden_conv,[POPULATION,1]), lambda: hidden_conv)
        
        sequence_output, state = tf.nn.dynamic_rnn(tf.contrib.rnn.LSTMCell(64),\
                                            tf.cast(self.genomes,tf.float32),dtype=tf.float32,sequence_length=length(self.genomes),scope=myScope+'_rnn')
        last = last_relevant(sequence_output, length(self.genomes)) 
        hidden_rnn= slim.fully_connected(last,64,biases_initializer=None,activation_fn=tf.nn.relu)
        hidden_rnn=tf.cond(self.condition < 1, lambda: hidden_rnn,lambda: tf.gather(hidden_rnn,self.correct_action)) 
        combined=tf.concat([hidden_rnn,hidden_conv],1)
        #
        #Change Recurrent Net to Match Inputs in Testing
        #hidden_rnn=tf.cond(self.condition < 1, lambda: hidden_rnn,lambda: tf.gather(hidden_rnn,self.action_holder)) 
        self.hidden_combined= slim.fully_connected(combined,h_size,biases_initializer=None,activation_fn=tf.nn.relu)
        self.hidden_combined_2= slim.fully_connected(self.hidden_combined,h_size//4,biases_initializer=None,activation_fn=tf.nn.relu)
        self.regression= slim.fully_connected(self.hidden_combined_2,1,biases_initializer=None,activation_fn=tf.nn.tanh)
        self.Advantage=tf.cond(self.condition <  1, 
        lambda: tf.reshape(self.regression,[1,POPULATION]), 
        lambda: tf.reshape(self.regression,[1,BATCH])[0])
        #self.Advantage=tf.reduce_mean(self.Advantage,axis=1,keep_dims=True)
        #Then combine them together to get our final Q-values.
        self.Mean=tf.reduce_mean(self.Advantage,axis=1,keep_dims=False)
        self.Qout=tf.cond(self.condition <  1, 
        lambda: self.Value + tf.subtract(self.Advantage,tf.reduce_mean(self.Advantage,axis=1,keep_dims=True)), 
        lambda: self.Value + self.Advantage-self.correct_mean)

        self.Smooth=tf.subtract(tf.reshape(self.Qout,[POPULATION]),tf.cast(self.used_genomes,tf.float32))
        self.predict = tf.argmax(self.Smooth,0)
        
        #Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
        self.targetQ = tf.placeholder(shape=[None],dtype=tf.float32)
        self.actions = tf.placeholder(shape=[None],dtype=tf.int32)
        #self.actions_onehot = tf.one_hot(self.actions,BATCH,dtype=tf.float32)
        
        #self.Q = tf.reduce_sum(tf.multiply(self.Qout, self.actions_onehot), axis=1)
        
        self.td_error = tf.square(self.targetQ - self.Qout)
        self.loss = tf.reduce_mean(self.td_error)
        self.trainer = tf.train.AdamOptimizer(learning_rate=0.0001)
        self.updateModel = self.trainer.minimize(self.loss)

### Checking network learning

Mean reward over time

In [ ]:
rMat = np.resize(np.array(rList),[len(rList)//100,100])
rMean = np.average(rMat,1)
plt.plot(rMean)

In [ ]:
trainBatch[:,2]

In [ ]:
targetQ

In [ ]:
arr = SQL.gain_history()
print(arr.shape)
x=np.random.shuffle(arr)
#print(arr)